# USDC Peg Deviation Analysis

This notebook analyzes hourly volume patterns outside the ±0.1% band for USDC/USDT across Uniswap V3 and Bybit.

## Analysis Goals
- Identify periods of USDC peg deviation
- Compare volume patterns between DEX and CEX
- Analyze price ranges during deviation periods
- Generate insights for market microstructure

## Data Sources
- **Uniswap V3**: USDC/USDT 0.01% pool (0x3416cf6c708da44db2624d63ea0aaef7113527c6)
- **Bybit**: USDC/USDT spot market
- **Timeframe**: July 1, 2025 - September 30, 2025 (UTC)
- **Band**: Outside ±0.1% around 1.0000 (price < 0.9990 or > 1.0010)


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timezone
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")


In [ ]:
# Load the aggregated data
df = pd.read_csv('outputs/usdc_peg_outside_band_hourly.csv')

print(f"Data shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"Date range: {df['time'].min()} to {df['time'].max()}")

# Convert time column to datetime
df['datetime'] = pd.to_datetime(df['time'])

# Display basic info
print("\nData Info:")
print(df.info())

# Show first few rows
print("\nFirst 10 rows:")
df.head(10)


In [ ]:
# Basic statistics and data quality checks
print("=== Data Quality Checks ===")

# Check for missing values
print("Missing values:")
print(df.isnull().sum())

# Check for negative volumes
print(f"\nNegative volumes:")
print(f"Uniswap: {(df['uniswap_volume'] < 0).sum()}")
print(f"Bybit: {(df['bybit_volume'] < 0).sum()}")

# Volume statistics
print(f"\nVolume Statistics:")
print(f"Uniswap total outside band: {df['uniswap_volume'].sum():,.2f} USDC")
print(f"Bybit total outside band: {df['bybit_volume'].sum():,.2f} USDC")

# Hours with activity
print(f"\nHours with outside-band activity:")
print(f"Uniswap: {(df['uniswap_volume'] > 0).sum()}")
print(f"Bybit: {(df['bybit_volume'] > 0).sum()}")

# Price range analysis
uniswap_active = df[df['uniswap_volume'] > 0]
bybit_active = df[df['bybit_volume'] > 0]

if not uniswap_active.empty:
    print(f"\nUniswap price range (outside band):")
    print(f"Min: {uniswap_active['uniswap_min_price'].min():.6f}")
    print(f"Max: {uniswap_active['uniswap_max_price'].max():.6f}")

if not bybit_active.empty:
    print(f"\nBybit price range (outside band):")
    print(f"Min: {bybit_active['bybit_min_price'].min():.6f}")
    print(f"Max: {bybit_active['bybit_max_price'].max():.6f}")


In [ ]:
# Plot 1: Hourly outside-band volume comparison
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Uniswap volume over time
ax1.plot(df['datetime'], df['uniswap_volume'], label='Uniswap V3', alpha=0.7)
ax1.set_title('Uniswap V3 Outside-Band Volume Over Time', fontsize=14, fontweight='bold')
ax1.set_ylabel('Volume (USDC)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Bybit volume over time
ax2.plot(df['datetime'], df['bybit_volume'], label='Bybit', alpha=0.7, color='orange')
ax2.set_title('Bybit Outside-Band Volume Over Time', fontsize=14, fontweight='bold')
ax2.set_ylabel('Volume (USDC)')
ax2.set_xlabel('Date')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Plot 2: Combined volume comparison
plt.figure(figsize=(15, 8))

# Plot both venues on same chart
plt.plot(df['datetime'], df['uniswap_volume'], label='Uniswap V3', alpha=0.7, linewidth=1)
plt.plot(df['datetime'], df['bybit_volume'], label='Bybit', alpha=0.7, linewidth=1)

plt.title('USDC Peg Deviation: Outside-Band Volume Comparison', fontsize=16, fontweight='bold')
plt.ylabel('Volume (USDC)')
plt.xlabel('Date')
plt.legend()
plt.grid(True, alpha=0.3)

# Add band boundaries as horizontal lines
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Plot 3: Price range analysis (scatter plot)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Uniswap price ranges
uniswap_active = df[df['uniswap_volume'] > 0]
if not uniswap_active.empty:
    ax1.scatter(uniswap_active['datetime'], uniswap_active['uniswap_min_price'], 
                alpha=0.6, label='Min Price', s=20)
    ax1.scatter(uniswap_active['datetime'], uniswap_active['uniswap_max_price'], 
                alpha=0.6, label='Max Price', s=20)
    ax1.axhline(y=0.9990, color='red', linestyle='--', alpha=0.7, label='Band Lower')
    ax1.axhline(y=1.0010, color='red', linestyle='--', alpha=0.7, label='Band Upper')
    ax1.set_title('Uniswap V3 Price Ranges (Outside Band)')
    ax1.set_ylabel('Price (USDT per USDC)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

# Bybit price ranges
bybit_active = df[df['bybit_volume'] > 0]
if not bybit_active.empty:
    ax2.scatter(bybit_active['datetime'], bybit_active['bybit_min_price'], 
                alpha=0.6, label='Min Price', s=20)
    ax2.scatter(bybit_active['datetime'], bybit_active['bybit_max_price'], 
                alpha=0.6, label='Max Price', s=20)
    ax2.axhline(y=0.9990, color='red', linestyle='--', alpha=0.7, label='Band Lower')
    ax2.axhline(y=1.0010, color='red', linestyle='--', alpha=0.7, label='Band Upper')
    ax2.set_title('Bybit Price Ranges (Outside Band)')
    ax2.set_ylabel('Price (USDT per USDC)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Plot 4: Volume distribution analysis
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Volume histograms
ax1.hist(df['uniswap_volume'], bins=50, alpha=0.7, label='Uniswap V3')
ax1.set_title('Uniswap V3 Volume Distribution')
ax1.set_xlabel('Volume (USDC)')
ax1.set_ylabel('Frequency')
ax1.legend()

ax2.hist(df['bybit_volume'], bins=50, alpha=0.7, label='Bybit', color='orange')
ax2.set_title('Bybit Volume Distribution')
ax2.set_xlabel('Volume (USDC)')
ax2.set_ylabel('Frequency')
ax2.legend()

# Log scale for better visualization
ax3.hist(df[df['uniswap_volume'] > 0]['uniswap_volume'], bins=50, alpha=0.7, label='Uniswap V3')
ax3.set_title('Uniswap V3 Volume Distribution (Log Scale)')
ax3.set_xlabel('Volume (USDC)')
ax3.set_ylabel('Frequency')
ax3.set_yscale('log')
ax3.legend()

ax4.hist(df[df['bybit_volume'] > 0]['bybit_volume'], bins=50, alpha=0.7, label='Bybit', color='orange')
ax4.set_title('Bybit Volume Distribution (Log Scale)')
ax4.set_xlabel('Volume (USDC)')
ax4.set_ylabel('Frequency')
ax4.set_yscale('log')
ax4.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Analysis: Correlation between venues
print("=== Correlation Analysis ===")

# Calculate correlation between venues
correlation = df['uniswap_volume'].corr(df['bybit_volume'])
print(f"Volume correlation between Uniswap and Bybit: {correlation:.4f}")

# Hours with activity in both venues
both_active = df[(df['uniswap_volume'] > 0) & (df['bybit_volume'] > 0)]
print(f"Hours with activity in both venues: {len(both_active)}")

# Hours with activity in only one venue
uniswap_only = df[(df['uniswap_volume'] > 0) & (df['bybit_volume'] == 0)]
bybit_only = df[(df['uniswap_volume'] == 0) & (df['bybit_volume'] > 0)]
print(f"Hours with Uniswap-only activity: {len(uniswap_only)}")
print(f"Hours with Bybit-only activity: {len(bybit_only)}")

# Total hours with any activity
any_activity = df[(df['uniswap_volume'] > 0) | (df['bybit_volume'] > 0)]
print(f"Total hours with any outside-band activity: {len(any_activity)}")
print(f"Percentage of hours with activity: {len(any_activity)/len(df)*100:.2f}%")


In [ ]:
# Final data preview
print("=== Final Data Preview ===")
print("Sample of the aggregated data:")
df.head()
